In [1]:
import numpy as np
from qutip import *
import dask
from dask import delayed
import dask.bag as db
from dask.distributed import Client, LocalCluster,progress
from tqdm import tqdm
import importlib
import functions_fbq
importlib.reload(functions_fbq)
from functions_fbq import *
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from figurestyle import *
figure_features()

In [8]:
from dask.distributed import Client

cliente = Client('tcp://132.166.20.131:8786')
cliente.upload_file("functions_fbq.py")

{'tcp://132.166.20.131:64085': {'status': 'OK'}}

In [11]:
Ec = 40e9
EDelta = 20e9
eigvals = 2
phi_ext = np.pi

El_list = 5 * 1e9 * np.logspace(-2, 1, 20)
r_list = np.linspace(0.05, 0.95, 20)
El_r_pairs = [(El, r) for El in El_list for r in r_list]

phi_matrix_elements_list = np.zeros((len(El_list), len(r_list)))
n_matrix_elements_list = np.zeros((len(El_list), len(r_list)))
dHdr_matrix_elements_list = np.zeros((len(El_list), len(r_list)))
d2E01dphi2_list = np.zeros((len(El_list), len(r_list)))

In [12]:
@delayed
def process_r(El, r):
    eigvals_list, _, matrix_op_sqr_list = eigensystem_and_matrix_elements_Josephson(Ec, El, EDelta, phi_ext, r,100, eigvals)
    phi_matrix_element = matrix_op_sqr_list[0]
    n_matrix_element = matrix_op_sqr_list[1]
    dH_dr_element = matrix_op_sqr_list[2]

    phi_ext_list = np.linspace(phi_ext, phi_ext +0.1, 10)
    eigvals_list = np.array([np.real(eigensystem_and_matrix_elements_Josephson(Ec, El, EDelta, phi_ext_aux, r, N = 100, eigvals = 2)[0][1] - eigensystem_and_matrix_elements_sqr_fbq(Ec, El, EDelta, 0, r, N = 50, eigvals = 2)[0][0]) for phi_ext_aux in phi_ext_list])

    phi_ext_list = np.concatenate([-phi_ext_list[::-1][0:-1], phi_ext_list])
    eigvals_list = np.concatenate([eigvals_list[::-1][0:-1], eigvals_list])

    dE01_dphi = np.gradient(eigvals_list, phi_ext_list, edge_order=1)
    d2E01_dphi2 = np.gradient(dE01_dphi, phi_ext_list, edge_order=1)
    d2E01dphi2 = d2E01_dphi2[len(phi_ext_list) // 2]

    return phi_matrix_element, n_matrix_element, dH_dr_element, d2E01dphi2

In [13]:
# Crear un dask bag a partir de la lista de tuplas y mapear la función process_r
results_bag = db.from_sequence(El_r_pairs).map(lambda x: process_r(x[0], x[1]))

# Calcular los resultados
computed_results = dask.compute(*results_bag)

# Asignar los resultados a las matrices correspondientes
k = 0
for i, El in enumerate(El_list):
    for j, r in enumerate(r_list):
        phi_matrix_element, n_matrix_element, dH_dr_element, d2E01dphi2 = computed_results[k]
        phi_matrix_elements_list[i, j] = El * np.abs(phi_matrix_element)
        n_matrix_elements_list[i, j] = 8 * Ec * np.abs(n_matrix_element)
        dHdr_matrix_elements_list[i, j] = np.abs(dH_dr_element)
        d2E01dphi2_list[i, j] = np.abs(d2E01dphi2)
        k += 1

In [14]:
cliente.close()